# Code to find outliers from MAGeT segmentation
## Uses brain masks from fMRIPrep and MAGeT segmentation volumes from CSVs (../results/) generated by "collect_volumes.sh" script part of [MAGeTBrain pipeline](https://github.com/CobraLab/documentation/wiki/MAGeTBrain#collecting-volumes)

In [11]:
import nibabel as nib
import glob
import os
import sys
import numpy as np
import pandas as pd

sys.path.append('../')
from lib.io_utils import *

### Paths

In [38]:
project_dir = '../../../../../'
data_dir = project_dir + 'data/preproc_T1s/'

cohort = 'PPMI'
masked_image_dir = data_dir + f'{cohort}/masked_images/'
maget_output = f'../results/{cohort}_volumes.csv'
participants_csv = f'../metadata/{cohort}_participants.txt'

print(f'Using {cohort} cohort')

Using PPMI cohort


## Read participants info

In [22]:
participants = pd.read_csv(participants_csv)
print(f'Found {len(participants)} participants')
participants.head()

Found 116 participants


,participants
0,sub-3000
1,sub-3004
2,sub-3008
3,sub-3011
4,sub-3013


### Grab TBV from brain masks

In [ ]:
mask_name_suffix = '_run-1_desc-masked_preproc_T1w.nii.gz'

brain_vol_list = []

for subject in participants['participants'].values:
    # print('Subject name: {}'.format(subject))
    mask_path = f'{masked_image_dir}{subject}{mask_name_suffix}' 
    mask = nib.load(mask_path).get_fdata()
    brain_vol = np.sum(mask>0)
    brain_vol_list.append(brain_vol)

vol_df = participants.copy()
vol_df['TBV'] = brain_vol_list
vol_df.head() 

In [46]:
## Read MAGeT volumes
cohort = 'PPMI'
maget_output = f'../results/{cohort}_volumes.csv'
maget_df = pd.read_csv(maget_output,sep=',',header=0)
maget_df

ParserError: Error tokenizing data. C error: Expected 27 fields in line 36, saw 28


In [ ]:
output/fusion/majority_vote/sub-3316_run-1_desc-masked_preproc_T1w_labels.mnc
output/fusion/majority_vote/sub-3519_run-1_desc-masked_preproc_T1w_labels.mnc
output/fusion/majority_vote/sub-3527_run-1_desc-masked_preproc_T1w_labels.mnc
output/fusion/majority_vote/sub-3803_run-1_desc-masked_preproc_T1w_labels.mnc
